In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from scipy import spatial
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
data= pd.read_csv('Books.csv', encoding="utf-8")
data

,Unnamed: 0,_id,author,contents,publishing house,year
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904


In [3]:
data['documents_cleaned']= data.apply(lambda row: nltk.sent_tokenize(row['contents']), axis=1)
data

,Unnamed: 0,_id,author,contents,publishing house,year,documents_cleaned
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ..."


In [4]:
import numpy as np
documents = list(np.array(data['documents_cleaned'][0]))
documents2 = list(np.array(data['documents_cleaned'][1]))

df = pd.DataFrame(documents,columns={'text'})
df2 = pd.DataFrame(documents2, columns={'text'})
df = df.append(df2)

#WORD2VEC

In [5]:
corpus_text = 'n'.join(data['contents'])
data = []
# iterate through each sentence in the file
for i in sent_tokenize(corpus_text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [6]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=5,
                                 window=10,
                                 size=50,
                                 sample=0,
                                 batch_words=100) 
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 50, window = 5, sg = 1)

In [7]:
#model2.train(data, total_examples=model2.corpus_count, epochs=60, report_delay=1)

In [8]:
model1.init_sims(replace=True)
model1.wv.most_similar(positive=["giorno"])

[('momento', 0.8715795874595642),
 ('matrimonio', 0.8673547506332397),
 ('tedesco', 0.8415037393569946),
 ('lume', 0.8407399654388428),
 ('galoppo', 0.8377944827079773),
 ('grido', 0.8357929587364197),
 ('pensiero', 0.8336276412010193),
 ('servo', 0.8306437730789185),
 ('passo', 0.8305108547210693),
 ('corpo', 0.8276510238647461)]

In [9]:
model2.init_sims(replace=True)

In [10]:
model2.wv.most_similar(positive=["giorno"])

[('anno', 0.893956184387207),
 ('mattina', 0.8754860162734985),
 ('durante', 0.8721182942390442),
 ('mattino', 0.8712909817695618),
 ('pomeriggio', 0.8689864873886108),
 ('periodo', 0.8669381737709045),
 ('seguente', 0.8663297891616821),
 ('passato', 0.8657146692276001),
 ('grido', 0.858664870262146),
 ('movimento', 0.8584903478622437)]

In [11]:
model2.wv.similarity('giorno','mattino')

0.8712909

In [12]:
similar = model2.wv.most_similar('mattino')
similar

[('pomeriggio', 0.9732072949409485),
 ('villaggio', 0.9599208831787109),
 ('termine', 0.9569495320320129),
 ('grido', 0.9537290930747986),
 ('buio', 0.9533131718635559),
 ('sonno', 0.9496479630470276),
 ('suono', 0.9490557909011841),
 ('terreno', 0.9464235901832581),
 ('lume', 0.9455164074897766),
 ('crepuscolo', 0.9434531927108765)]

In [13]:
model2.init_sims(replace = True)
model2.save('word2vec_model')
model = Word2Vec.load('word2vec_model')
model.wv.most_similar('pomeriggio')

[('mattino', 0.9732073545455933),
 ('lume', 0.9651427268981934),
 ('salotto', 0.9638891220092773),
 ('capo', 0.9632396697998047),
 ('villaggio', 0.9626756906509399),
 ('termine', 0.9553670287132263),
 ('vassoio', 0.9548516869544983),
 ('suonò', 0.9541739225387573),
 ('campanello', 0.9535472393035889),
 ('viaggio', 0.9522131085395813)]

In [14]:
model.wv.most_similar('mattino')

[('pomeriggio', 0.9732072949409485),
 ('villaggio', 0.9599208831787109),
 ('termine', 0.9569495320320129),
 ('grido', 0.9537292122840881),
 ('buio', 0.9533131718635559),
 ('sonno', 0.9496480822563171),
 ('suono', 0.9490559101104736),
 ('terreno', 0.9464235901832581),
 ('lume', 0.9455164074897766),
 ('crepuscolo', 0.9434532523155212)]

In [15]:
distance = model.wv.wmdistance(data[0],data[12])
distance

0.7196691074362923

In [16]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.044275334954232186

In [17]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.46285087874550035

In [18]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.15785662684273333

In [19]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.07273662066734342

#DOC2VEC

In [20]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df.text)]
model_d2v = Doc2Vec(vector_size=5, dbow_words=1, dm=0, epochs=30,  window=2, seed=1337, min_count=1, alpha=0.001, min_alpha=0.025)
model_d2v.build_vocab(tagged_data)
model_d2v.save('doc2vec_model')

In [21]:
model_d2v = Doc2Vec(vector_size=5, window=2, min_count=1, workers=4)
model_d2v.build_vocab(tagged_data)

In [22]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(7888, 0.9923692345619202),
 (10840, 0.9826305508613586),
 (11627, 0.9826103448867798),
 (6849, 0.9799116253852844),
 (2725, 0.9785209894180298),
 (10862, 0.9783556461334229),
 (17309, 0.9774425625801086),
 (2840, 0.9747253060340881),
 (16098, 0.9731336236000061),
 (17352, 0.972244381904602)]

In [23]:
tagged_data[1479]

TaggedDocument(words=['E', 'come', 'tali', 'devo', 'trattarvi', "''", '.'], tags=[1479])

In [24]:
tagged_data[4549]

TaggedDocument(words=['La', 'Sorte', 'ti', 'ha', 'assegnato', 'una', 'dose', 'di', 'felicità', ',', 'questo', 'lo', 'so', '.'], tags=[4549])

In [25]:
tokens = 'Quel giorno era impossibile uscire a passeggio.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(7755, 0.9911394715309143),
 (4708, 0.9905685186386108),
 (13370, 0.9882455468177795),
 (19003, 0.9874914288520813),
 (10470, 0.986694872379303),
 (9452, 0.9864989519119263),
 (1042, 0.9848476648330688),
 (6487, 0.9807798862457275),
 (7645, 0.9807083010673523),
 (12488, 0.9807029366493225)]

In [26]:
tagged_data[5891]

TaggedDocument(words=['Ascol-tate', '!', "''"], tags=[5891])

In [27]:
tagged_data[18731]

TaggedDocument(words=['``', "Sull'altare", 'deporrò', 'il', 'cuore', ',', 'la', 'vita', ',', 'la', 'vittima', 'intera', '.'], tags=[18731])

In [28]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(7888, 0.9923692345619202),
 (10840, 0.9826305508613586),
 (11627, 0.9826103448867798),
 (6849, 0.9799116253852844),
 (2725, 0.9785209894180298),
 (10862, 0.9783556461334229),
 (17309, 0.9774425625801086),
 (2840, 0.9747253060340881),
 (16098, 0.9731336236000061),
 (17352, 0.972244381904602)]

In [29]:
tagged_data[1479]

TaggedDocument(words=['E', 'come', 'tali', 'devo', 'trattarvi', "''", '.'], tags=[1479])

In [30]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.01226409403765839

In [31]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
new_vector_a = model_d2v.infer_vector(sentence_a)
new_vector_b = model_d2v.infer_vector(sentence_b)

cosine_similarity = np.dot(new_vector_a, new_vector_b)/(np.linalg.norm(new_vector_a)* np.linalg.norm(new_vector_b))
cosine_similarity

-0.37678933

In [32]:
score = model_d2v.docvecs.most_similar([model_d2v.infer_vector(data[0])],topn=40)
score

[(13673, 0.9892325401306152),
 (15596, 0.988711416721344),
 (14253, 0.9860968589782715),
 (12970, 0.975676417350769),
 (6942, 0.9731554985046387),
 (10038, 0.9683110117912292),
 (12451, 0.9675349593162537),
 (13915, 0.966421902179718),
 (9123, 0.9558419585227966),
 (5643, 0.9554709196090698),
 (19396, 0.9528900980949402),
 (14408, 0.9510817527770996),
 (19158, 0.947481632232666),
 (15870, 0.945288896560669),
 (13858, 0.9450591206550598),
 (2337, 0.942715048789978),
 (1712, 0.9417337775230408),
 (12183, 0.9396962523460388),
 (1210, 0.9383390545845032),
 (6199, 0.9370242357254028),
 (5941, 0.9367942214012146),
 (14527, 0.9345465302467346),
 (4993, 0.9341862797737122),
 (11098, 0.9326948523521423),
 (17864, 0.9297665357589722),
 (8033, 0.929499089717865),
 (14434, 0.9293380975723267),
 (3716, 0.9289158582687378),
 (16262, 0.9247530102729797),
 (8981, 0.9241350293159485),
 (10324, 0.9240264296531677),
 (3927, 0.9234679937362671),
 (11849, 0.9227210879325867),
 (9772, 0.922474205493927),
 (

In [33]:
import numpy as np
def cos_sim(a, b):
  dot_product = np.dot(a, b)
  norm_a = np.linalg.norm(a)
  norm_b = np.linalg.norm(b)
  return dot_product / (norm_a * norm_b)


print(cos_sim(new_vector_a,new_vector_b))

-0.37678933


In [34]:
cos_distance = spatial.distance.cosine(new_vector_a, new_vector_b)
cos_distance

1.3767893314361572

In [35]:
#Euclidean similarity
euclidean_similarity = 1 / (1 + (np.linalg.norm(new_vector_a-new_vector_b)))
euclidean_similarity

0.8318079626701173

#SENT2VEC

In [36]:
!pip install bert
!pip install pytorch_pretrained_bert
!pip install sent2vec
!pip install sentence_transformers

  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3766 sha256=51ac4a57553be954508be923e2416f2d4919d8373592486ce6db0247ab2ac3c6
  Stored in directory: /root/.cache/pip/wheels/bb/31/1b/c05f362e347429b7436954d1a2280fe464731e8f569123a848
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6795 sha256=d84b1984a6415cb2f6a268bf6350b45e5804df8fcf8dec4611f4838594ae8348
  Stored in directory: /root/.cache/pip/wheels/94/f1/b4/0b98b1e94775da6a0b1130e342d22af05cd269e1172c19f40f
Successfully built bert erlastic
     |████████████████████████████████| 123 kB 2.1 MB/s 
     |████████████████████████████████| 131 kB 8.0 MB/s 
     |████████████████████████████████| 79 kB 4.3 MB/s 
     |████████████████████████████████| 8.0 MB 10.8 MB/s 
     |████████████████████████████████| 138 kB 34.9 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 78 kB 2.2 MB/s 
     |████████████████████████████████| 1.2 MB 5.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=6912ba58dfaf4d1937e7ad23ac8e8702f0318f0629018bd23cfcb1a756d8ce9b
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [37]:
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import random
import numpy as np
import matplotlib.pyplot as plt
import math


In [38]:
!pip install sentence_transformers

In [56]:
from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [57]:
# Two lists of sentences
sentences1 = documents

sentences2 = documents2

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

In [58]:
#Output the pairs with their score
for i in range(len(sentences1[:30])):
      print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[0], sentences2[i], cosine_scores[i][i]))

Quel giorno era impossibile uscire a passeggio. 		 In quel giorno era impossibile passeggiare. 		 Score: 0.9490
Quel giorno era impossibile uscire a passeggio. 		 La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione. 		 Score: 0.8784
Quel giorno era impossibile uscire a passeggio. 		 Ne ero contenta. 		 Score: 0.9129
Quel giorno era impossibile uscire a passeggio. 		 Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed. 		 Score: 0.3790
Quel giorno era impossibile uscire a passeggio. 

In [55]:
document_embedding = model.encode(data)

In [40]:
query_embedding = model.encode(['Quel giorno era impossibile uscire a passeggio.'])
passage_embedding = model.encode(['In quel giorno era impossibile passeggiare.'])

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.9490]])

In [41]:
query_embedding = model.encode(['Ne fui felice.'])

passage_embedding = model.encode(['Ne ero contenta.'])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.9129]])

In [44]:
query_embedding = model.encode(['Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'])

passage_embedding = model.encode(['Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.8191]])

In [45]:
query_embedding = model.encode(['Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'])
passage_embedding = model.encode(['Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.7208]])

In [48]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = documents

sentences2 = documents2

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [49]:
#Output the pairs with their score
for i in range(len(sentences1[:30])):
      print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[0], sentences2[i], cosine_scores[i][i]))

Quel giorno era impossibile uscire a passeggio. 		 In quel giorno era impossibile passeggiare. 		 Score: 0.9375
Quel giorno era impossibile uscire a passeggio. 		 La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione. 		 Score: 0.7308
Quel giorno era impossibile uscire a passeggio. 		 Ne ero contenta. 		 Score: 0.5094
Quel giorno era impossibile uscire a passeggio. 		 Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed. 		 Score: 0.6617
Quel giorno era impossibile uscire a passeggio. 